In [10]:
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
from scipy.optimize import curve_fit
from uncertainties import ufloat
from uncertainties import unumpy as unp
import array_to_latex as a2l
import csv
import pandas as pd
from astropy.io.votable import parse
from astropy.table import QTable, Table, Column
import sympy as sym

rc('font', **{'family': 'serif', 'serif': ['Computer Modern']}) #font na grafih je LaTexov
rc('text', usetex=True)

In [11]:
# kocka
a, Δa = 60e-3, 0.5e-3
d, Δd = 40e-3, 0.5e-3
m_k, dm_k = 1193e-3, 1e-3

# valj
r, Δr = 15e-3 / 2, 0.5e-3 / 2
R, ΔR = 87.2e-3 / 2, 0.1e-3 / 2
H, ΔH = 50e-3, 0.5e-3
m_v, Δm_v = 2500e-3, 1e-3

ℓ, Δℓ = 290e-3, 5e-3
ρ, Δρ = 0.63e-3, 0.05e-3

m_z, Δm_z = 754e-3, 2e-3

t_p = 1/10 * np.array([18.972, 18.744, 18.875])
σt_p = np.std(t_p)
t_v = 1/10 * np.array([52.996, 52.625, 53.169])
σt_v = np.std(t_v)
t_k = 1/10 * np.array([36.361, 36.660, 36.240])
σt_k = np.std(t_k)
t_z = 1/10 * np.array([28.954, 29.433, 29.857])
σt_z = np.std(t_z)
print(f'{σt_p:.3f} & {σt_v:.3f} & {σt_k:.3f} & {σt_z:.3f} \\\\\n')

0.009 & 0.023 & 0.018 & 0.037 \\



In [12]:
print(str.join('',
    [f'{t_p:.3f} & {t_v:.3f} & {t_k:.3f} & {t_z:.3f} \\\\\n'
     for t_p, t_v, t_k, t_z in sorted(zip(t_p, t_v, t_k, t_z))]
))

1.874 & 5.263 & 3.666 & 2.943 \\
1.888 & 5.317 & 3.624 & 2.986 \\
1.897 & 5.300 & 3.636 & 2.895 \\



In [13]:
t_p = np.mean(t_p)
t_v = np.mean(t_v)
t_k = np.mean(t_k)
t_z = np.mean(t_z)
print(f'{t_p:.3f} & {t_v:.3f} & {t_k:.3f} & {t_z:.3f} \\\\\n')

1.886 & 5.293 & 3.642 & 2.941 \\



In [39]:
# vztrajnostni moment valja 
J_v = 0.5*m_v * (r**2 + R**2)
ΔJ_v = np.sqrt(
    (0.5*m_v * 2*r * Δr)**2 +
    (0.5*m_v * 2*R * ΔR)**2
)
1e3 * J_v, 1e3 * ΔJ_v

(2.4465125000000003, 0.007188543402525995)

In [43]:
 _J_v, _t_v, _t_p = sym.symbols('J_v t_v t_p')
num_assoc = {_J_v: J_v, _t_v: t_v, _t_p: t_p}
_D = _J_v * (2*sym.pi / _t_v)**2 / (1 - (_t_p/_t_v)**2)
_ΔD = sym.sqrt(
    (_D.diff(_J_v) * ΔJ_v)**2 +
    (_D.diff(_t_v) * σt_v)**2 +
    (_D.diff(_t_p) * σt_p)**2
)

D = _D.evalf(10, subs=num_assoc)
ΔD = _ΔD.evalf(10, subs=num_assoc)
1e3 * D, 1e3 * ΔD

(3.94907743702788, 0.0408865926071478)

In [44]:
# Calculate J_μ for body μ. The actual data to be used is given as values in the `num_assoc` dict.
_D, _t_μ, _σt_μ, _t_p = sym.symbols('D t_\mu \sigma_{t_μ} t_p')
num_assoc = {_D: D, _t_μ: t_k, _σt_μ: σt_k, _t_p: t_p}
_J_μ = _D * (1 - (_t_p/_t_μ)**2) * (_t_μ / (2*sym.pi))**2
_ΔJ_μ = sym.sqrt(
    (_J_μ.diff(_D) * ΔD)**2 +
    (_J_μ.diff(_t_μ) * _σt_μ)**2 +
    (_J_μ.diff(_t_p) * σt_p)**2
)

J_μ = _J_μ.evalf(10, subs=num_assoc)
ΔJ_μ = _ΔJ_μ.evalf(10, subs=num_assoc)
1e3 * J_μ, 1e3 * ΔJ_μ

(0.970906559182083, 0.0167006235589628)

In [46]:
 _m_k, _a, _d = sym.symbols('m_k a d')
num_assoc = {_m_k: m_k, _a: a, _d: d}
_J_k = _m_k * (2/3 * _a**4 - sym.pi/8 * _d**4) / (4*_a**2 - sym.pi * _d**2)
_ΔJ_k = sym.sqrt(
    (_J_k.diff(_m_k) * dm_k)**2 +
    (_J_k.diff(_a) * Δa)**2 +
    (_J_k.diff(_d) * Δd)**2
)

_J_k = _J_k.evalf(10, subs=num_assoc)
_ΔJ_k = _ΔJ_k.evalf(10, subs=num_assoc)
1e3 * _J_k, 1e3 * _ΔJ_k

(0.971700268735276, 0.0135378140730946)